# Het Rank - 202301081
##IT-549 LAB-2

##Setup

In [1]:
!pip install kaggle
!pip install nltk

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import mean_squared_error, f1_score, hamming_loss, jaccard_score
from tqdm import tqdm

In [9]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
df = pd.read_excel("/content/movies.xlsx")
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [11]:
df = df[['overview','tagline','keywords','genres','vote_average']]
df = df.dropna(subset=['vote_average'])
df.head()

,overview,tagline,keywords,genres,vote_average
0,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,culture clash future space war space colony so...,Action Adventure Fantasy Science Fiction,7.2
1,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",ocean drug abuse exotic island east india trad...,Adventure Fantasy Action,6.9
2,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,spy based on novel secret agent sequel mi6,Action Adventure Crime,6.3
3,Following the death of District Attorney Harve...,The Legend Ends,dc comics crime fighter terrorist secret ident...,Action Crime Drama Thriller,7.6
4,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",based on novel mars medallion space travel pri...,Action Adventure Science Fiction,6.1


# Text Cleaning

In [12]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):

    if pd.isna(text):
        return ""

    text = text.lower()

    text = re.sub(r"http\S+|www\S+|https\S+", '', text)

    text = re.sub(r'\d+', '', text)

    text = re.sub(r'[^\w\s]', '', text)

    tokens = word_tokenize(text)

    tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word not in stop_words and len(word) > 2
    ]

    return " ".join(tokens)

In [13]:
df['overview_clean'] = df['overview'].apply(clean_text)
df['tagline_clean'] = df['tagline'].apply(clean_text)
df['keywords_clean'] = df['keywords'].apply(clean_text)

#1.   Train Test Split
#2.   TF-IDF Vectorizing
#3.   GloVe loading



In [14]:
train_df, temp_df = train_test_split(df, test_size=0.30, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.50, random_state=42)

print(len(train_df), len(val_df), len(test_df))

3362 720 721


In [15]:
tfidf = TfidfVectorizer(max_features=20000)
tfidf.fit(train_df['overview_clean'])

TfidfVectorizer(max_features=20000)

In [16]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2026-02-20 11:24:13--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2026-02-20 11:24:14--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2026-02-20 11:24:14--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

**we are using glove.6B.100d.txt**

In [17]:
def load_glove(path):
    glove = {}

    with open(path, 'r', encoding='utf8') as f:
        for line in tqdm(f):
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            glove[word] = vector

    return glove

glove = load_glove('/content/glove.6B.100d.txt')

400000it [00:12, 31610.19it/s]


## Coverage of the Embeddings

In [18]:
vocab = tfidf.vocabulary_
covered = 0

for word in vocab:
    if word in glove:
        covered += 1

coverage = covered / len(vocab)
print("Embedding coverage:", coverage)

Embedding coverage: 0.8922467325093145


# Creating Document Embeddings

In [19]:
def document_embedding(text):

    words = text.split()
    vec = np.zeros(100)
    weight_sum = 0

    tfidf_scores = tfidf.transform([text])

    for word in words:
        if word in glove and word in tfidf.vocabulary_:
            idx = tfidf.vocabulary_[word]
            weight = tfidf_scores[0, idx]

            vec += weight * glove[word]
            weight_sum += weight

    if weight_sum != 0:
        vec = vec / weight_sum

    return vec

In [51]:
X_train = np.vstack(train_df['overview_clean'].apply(document_embedding))
X_val   = np.vstack(val_df['overview_clean'].apply(document_embedding))
X_test  = np.vstack(test_df['overview_clean'].apply(document_embedding))

# A.Rating Prediction (Regression)

In [21]:
y_train = train_df['vote_average'].values.astype(np.float32)
y_val   = val_df['vote_average'].values.astype(np.float32)
y_test  = test_df['vote_average'].values.astype(np.float32)

In [22]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
X_val_t   = torch.tensor(X_val, dtype=torch.float32)
X_test_t  = torch.tensor(X_test, dtype=torch.float32)

y_train_t = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_val_t   = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)
y_test_t  = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [23]:
class RatingRegressor(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(100, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),

            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)

In [24]:
model = RatingRegressor()

In [25]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
epochs = 75

for epoch in range(epochs):

    model.train()
    optimizer.zero_grad()

    preds = model(X_train_t)
    loss = criterion(preds, y_train_t)

    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_preds = model(X_val_t)
        val_loss = criterion(val_preds, y_val_t)

    print(f"Epoch {epoch+1}: Train Loss={loss.item():.4f} | Val Loss={val_loss.item():.4f}")

Epoch 1: Train Loss=2.5106 | Val Loss=2.7271
Epoch 2: Train Loss=2.4616 | Val Loss=2.6736
Epoch 3: Train Loss=2.4575 | Val Loss=2.6206
Epoch 4: Train Loss=2.4113 | Val Loss=2.5711
Epoch 5: Train Loss=2.4132 | Val Loss=2.5262
Epoch 6: Train Loss=2.4142 | Val Loss=2.4874
Epoch 7: Train Loss=2.4369 | Val Loss=2.4551
Epoch 8: Train Loss=2.3930 | Val Loss=2.4289
Epoch 9: Train Loss=2.3841 | Val Loss=2.4098
Epoch 10: Train Loss=2.4102 | Val Loss=2.3967
Epoch 11: Train Loss=2.3911 | Val Loss=2.3887
Epoch 12: Train Loss=2.3527 | Val Loss=2.3858
Epoch 13: Train Loss=2.4077 | Val Loss=2.3859
Epoch 14: Train Loss=2.3663 | Val Loss=2.3890
Epoch 15: Train Loss=2.2950 | Val Loss=2.3949
Epoch 16: Train Loss=2.3354 | Val Loss=2.4030
Epoch 17: Train Loss=2.3508 | Val Loss=2.4127
Epoch 18: Train Loss=2.3297 | Val Loss=2.4220
Epoch 19: Train Loss=2.3218 | Val Loss=2.4303
Epoch 20: Train Loss=2.3577 | Val Loss=2.4352
Epoch 21: Train Loss=2.3104 | Val Loss=2.4366
Epoch 22: Train Loss=2.3099 | Val Loss=2.43

In [33]:
mean_rating = y_train.mean()
baseline_preds = np.full_like(y_test, mean_rating)

baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_preds))
print("Baseline RMSE:", baseline_rmse)

Baseline RMSE: 1.132481072711884


In [34]:
model.eval()
with torch.no_grad():
    test_preds = model(X_test_t).numpy()

mse = mean_squared_error(y_test, test_preds)
rmse = np.sqrt(mse)

print("Model MSE:", mse)
print("Model RMSE:", rmse)

Model MSE: 1.5697038173675537
Model RMSE: 1.252878213302296


**Performing regression task on "Tagline"**

In [36]:
X_train_2 = np.vstack(train_df['tagline_clean'].apply(document_embedding))
X_val_2   = np.vstack(val_df['tagline_clean'].apply(document_embedding))
X_test_2  = np.vstack(test_df['tagline_clean'].apply(document_embedding))

In [37]:
y_train_2 = train_df['vote_average'].values.astype(np.float32)
y_val_2   = val_df['vote_average'].values.astype(np.float32)
y_test_2  = test_df['vote_average'].values.astype(np.float32)

In [45]:
X_train_t_2 = torch.tensor(X_train_2, dtype=torch.float32)
X_val_t_2   = torch.tensor(X_val_2, dtype=torch.float32)
X_test_t_2  = torch.tensor(X_test_2, dtype=torch.float32)

y_train_t_2 = torch.tensor(y_train_2, dtype=torch.float32).unsqueeze(1)
y_val_t_2   = torch.tensor(y_val_2, dtype=torch.float32).unsqueeze(1)
y_test_t_2  = torch.tensor(y_test_2, dtype=torch.float32).unsqueeze(1)

In [46]:
model_2 = RatingRegressor()

In [47]:
criterion_2 = nn.MSELoss()
optimizer_2 = optim.Adam(model.parameters(), lr=0.001)

In [48]:
epochs = 75

for epoch in range(epochs):

    model_2.train()
    optimizer_2.zero_grad()

    preds = model_2(X_train_t_2)
    loss = criterion_2(preds, y_train_t_2)

    loss.backward()
    optimizer_2.step()

    model.eval()
    with torch.no_grad():
        val_preds = model_2(X_val_t_2)
        val_loss = criterion_2(val_preds, y_val_t_2)

    print(f"Epoch {epoch+1}: Train Loss={loss.item():.4f} | Val Loss={val_loss.item():.4f}")

Epoch 1: Train Loss=40.1921 | Val Loss=40.0649
Epoch 2: Train Loss=40.1919 | Val Loss=40.0320
Epoch 3: Train Loss=40.2009 | Val Loss=40.0513
Epoch 4: Train Loss=40.1895 | Val Loss=40.0581
Epoch 5: Train Loss=40.1952 | Val Loss=40.0525
Epoch 6: Train Loss=40.1941 | Val Loss=40.0499
Epoch 7: Train Loss=40.1926 | Val Loss=40.0576
Epoch 8: Train Loss=40.1973 | Val Loss=40.0645
Epoch 9: Train Loss=40.1894 | Val Loss=40.0498
Epoch 10: Train Loss=40.1897 | Val Loss=40.0622
Epoch 11: Train Loss=40.1966 | Val Loss=40.0422
Epoch 12: Train Loss=40.1921 | Val Loss=40.0775
Epoch 13: Train Loss=40.1974 | Val Loss=40.0529
Epoch 14: Train Loss=40.1909 | Val Loss=40.0461
Epoch 15: Train Loss=40.1975 | Val Loss=40.0563
Epoch 16: Train Loss=40.1936 | Val Loss=40.0580
Epoch 17: Train Loss=40.1943 | Val Loss=40.0455
Epoch 18: Train Loss=40.1943 | Val Loss=40.0722
Epoch 19: Train Loss=40.1833 | Val Loss=40.0616
Epoch 20: Train Loss=40.1951 | Val Loss=40.0439
Epoch 21: Train Loss=40.1963 | Val Loss=40.0555
E

In [49]:
mean_rating = y_train_2.mean()
baseline_preds = np.full_like(y_test_2, mean_rating)

baseline_rmse = np.sqrt(mean_squared_error(y_test_2, baseline_preds))
print("Baseline RMSE:", baseline_rmse)

Baseline RMSE: 1.132481072711884


In [50]:
model.eval()
with torch.no_grad():
    test_preds = model(X_test_t_2).numpy()

mse = mean_squared_error(y_test_2, test_preds)
rmse = np.sqrt(mse)

print("Model MSE:", mse)
print("Model RMSE:", rmse)

Model MSE: 7.283371925354004
Model RMSE: 2.698772299649232


# B.Genre Prediction (Multi-Label Classification)

## Genres into labels

In [59]:
def extract_genres(text):
    if pd.isna(text) or text == "":
        return []
    # Split the string by spaces and return as a list
    return text.split()

train_df['genre_list'] = train_df['genres'].apply(extract_genres)
val_df['genre_list']   = val_df['genres'].apply(extract_genres)
test_df['genre_list']  = test_df['genres'].apply(extract_genres)

train_df['genre_list'].head(10)

,genre_list
2874,"[Mystery, Thriller]"
2964,"[Science, Fiction, Thriller, Horror]"
3434,"[Romance, Drama]"
2118,"[Thriller, Drama, Mystery]"
4241,[Comedy]
1469,"[Action, Comedy, Thriller]"
1237,[Comedy]
103,"[Fantasy, Adventure, Action, Comedy, Drama]"
4758,"[Thriller, Science, Fiction]"
2274,[Thriller]


In [60]:
train_df = train_df[train_df['genre_list'].map(len) > 0]
val_df   = val_df[val_df['genre_list'].map(len) > 0]
test_df  = test_df[test_df['genre_list'].map(len) > 0]

print(len(train_df), len(val_df), len(test_df))

3344 712 719


## Multi-label Encoding

In [61]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
Y_train = mlb.fit_transform(train_df['genre_list'])
Y_val   = mlb.transform(val_df['genre_list'])
Y_test  = mlb.transform(test_df['genre_list'])

print("Genres:", mlb.classes_)
print("Number of genres:", len(mlb.classes_))

Genres: ['Action' 'Adventure' 'Animation' 'Comedy' 'Crime' 'Documentary' 'Drama'
 'Family' 'Fantasy' 'Fiction' 'Foreign' 'History' 'Horror' 'Movie' 'Music'
 'Mystery' 'Romance' 'Science' 'TV' 'Thriller' 'War' 'Western']
Number of genres: 22


In [62]:
Y_train_t = torch.tensor(Y_train, dtype=torch.float32)
Y_val_t   = torch.tensor(Y_val, dtype=torch.float32)
Y_test_t  = torch.tensor(Y_test, dtype=torch.float32)

In [78]:
X_train = np.vstack(train_df['overview_clean'].apply(document_embedding))
X_val   = np.vstack(val_df['overview_clean'].apply(document_embedding))
X_test  = np.vstack(test_df['overview_clean'].apply(document_embedding))

X_train_t = torch.tensor(X_train, dtype=torch.float32)
X_val_t   = torch.tensor(X_val, dtype=torch.float32)
X_test_t  = torch.tensor(X_test, dtype=torch.float32)

## Neural Network for Multi-Label

In [64]:
num_genres = Y_train.shape[1]

class GenreClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(100, 128),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(128, 64),
            nn.ReLU(),

            nn.Linear(64, num_genres)
        )

    def forward(self, x):
        return self.net(x)

In [65]:
genre_model = GenreClassifier()

In [66]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(genre_model.parameters(), lr=0.001)

In [67]:
epochs = 50

for epoch in range(epochs):

    genre_model.train()
    optimizer.zero_grad()

    outputs = genre_model(X_train_t)
    loss = criterion(outputs, Y_train_t)

    loss.backward()
    optimizer.step()

    # validation
    genre_model.eval()
    with torch.no_grad():
        val_out = genre_model(X_val_t)
        val_loss = criterion(val_out, Y_val_t)

    print(f"Epoch {epoch+1}: Train={loss.item():.4f} | Val={val_loss.item():.4f}")

Epoch 1: Train=0.6914 | Val=0.6873
Epoch 2: Train=0.6875 | Val=0.6833
Epoch 3: Train=0.6836 | Val=0.6791
Epoch 4: Train=0.6796 | Val=0.6748
Epoch 5: Train=0.6753 | Val=0.6701
Epoch 6: Train=0.6707 | Val=0.6650
Epoch 7: Train=0.6656 | Val=0.6591
Epoch 8: Train=0.6598 | Val=0.6522
Epoch 9: Train=0.6532 | Val=0.6441
Epoch 10: Train=0.6456 | Val=0.6347
Epoch 11: Train=0.6368 | Val=0.6240
Epoch 12: Train=0.6271 | Val=0.6119
Epoch 13: Train=0.6159 | Val=0.5982
Epoch 14: Train=0.6032 | Val=0.5830
Epoch 15: Train=0.5895 | Val=0.5663
Epoch 16: Train=0.5739 | Val=0.5481
Epoch 17: Train=0.5573 | Val=0.5287
Epoch 18: Train=0.5386 | Val=0.5083
Epoch 19: Train=0.5197 | Val=0.4871
Epoch 20: Train=0.4995 | Val=0.4655
Epoch 21: Train=0.4782 | Val=0.4439
Epoch 22: Train=0.4575 | Val=0.4229
Epoch 23: Train=0.4359 | Val=0.4030
Epoch 24: Train=0.4162 | Val=0.3847
Epoch 25: Train=0.3974 | Val=0.3685
Epoch 26: Train=0.3802 | Val=0.3551
Epoch 27: Train=0.3651 | Val=0.3446
Epoch 28: Train=0.3525 | Val=0.3372
E

In [68]:
genre_model.eval()
with torch.no_grad():
    logits = genre_model(X_test_t)
    probs = torch.sigmoid(logits)
    preds = (probs > 0.5).int().numpy()

In [69]:
from sklearn.metrics import f1_score, hamming_loss, jaccard_score

micro_f1 = f1_score(Y_test, preds, average='micro')
macro_f1 = f1_score(Y_test, preds, average='macro')
hamm = hamming_loss(Y_test, preds)
jacc = jaccard_score(Y_test, preds, average='samples')

print("Micro F1:", micro_f1)
print("Macro F1:", macro_f1)
print("Hamming Loss:", hamm)
print("Jaccard:", jacc)

Micro F1: 0.19982547993019198
Macro F1: 0.028440139095876803
Hamming Loss: 0.11594386142369452
Jaccard: 0.1567686601761706


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Frequent Words per Genre:

In [70]:
from collections import defaultdict

genre_docs = defaultdict(list)

for _, row in train_df.iterrows():
    text = row['overview_clean']
    for g in row['genre_list']:
        genre_docs[g].append(text)

In [71]:
from collections import Counter

genre_word_counts = {}

for genre, docs in genre_docs.items():

    counter = Counter()

    for doc in docs:
        words = doc.split()
        counter.update(words)

    genre_word_counts[genre] = counter

In [72]:
for genre in genre_word_counts:
    print("\nGENRE:", genre)
    print("Top words:", genre_word_counts[genre].most_common(10))


GENRE: Mystery
Top words: [('find', 51), ('life', 48), ('one', 42), ('murder', 33), ('new', 30), ('year', 30), ('world', 29), ('man', 28), ('young', 27), ('begin', 23)]

GENRE: Thriller
Top words: [('life', 164), ('find', 150), ('one', 134), ('new', 124), ('man', 109), ('young', 108), ('world', 103), ('two', 93), ('year', 91), ('get', 90)]

GENRE: Science
Top words: [('world', 95), ('find', 74), ('one', 62), ('earth', 62), ('year', 61), ('alien', 61), ('life', 54), ('time', 54), ('planet', 54), ('new', 53)]

GENRE: Fiction
Top words: [('world', 95), ('find', 74), ('one', 62), ('earth', 62), ('year', 61), ('alien', 61), ('life', 54), ('time', 54), ('planet', 54), ('new', 53)]

GENRE: Horror
Top words: [('one', 58), ('year', 58), ('new', 58), ('group', 55), ('find', 54), ('life', 52), ('young', 50), ('friend', 48), ('family', 43), ('world', 42)]

GENRE: Romance
Top words: [('life', 198), ('love', 191), ('young', 120), ('find', 112), ('woman', 105), ('man', 104), ('new', 101), ('one', 96

In [73]:
for genre, counter in genre_word_counts.items():

    rare_words = [(w,c) for w,c in counter.items() if c >= 3]
    rare_words = sorted(rare_words, key=lambda x: x[1])[:10]

    print("\nGENRE:", genre)
    print("Rare words:", rare_words)


GENRE: Mystery
Rare words: [('awakens', 3), ('next', 3), ('link', 3), ('teenager', 3), ('rock', 3), ('instead', 3), ('psychological', 3), ('america', 3), ('pressure', 3), ('top', 3)]

GENRE: Thriller
Rare words: [('mar', 3), ('ground', 3), ('vanishes', 3), ('morning', 3), ('chased', 3), ('foreign', 3), ('commit', 3), ('jackson', 3), ('fisherman', 3), ('ambassador', 3)]

GENRE: Science
Rare words: [('base', 3), ('collect', 3), ('collapse', 3), ('without', 3), ('drawn', 3), ('isolated', 3), ('jurassic', 3), ('genetically', 3), ('hammond', 3), ('document', 3)]

GENRE: Fiction
Rare words: [('base', 3), ('collect', 3), ('collapse', 3), ('without', 3), ('drawn', 3), ('isolated', 3), ('jurassic', 3), ('genetically', 3), ('hammond', 3), ('document', 3)]

GENRE: Horror
Rare words: [('made', 3), ('discovery', 3), ('collect', 3), ('without', 3), ('rule', 3), ('doesnt', 3), ('faith', 3), ('investigator', 3), ('eight', 3), ('john', 3)]

GENRE: Romance
Rare words: [('tension', 3), ('sergeant', 3), 

# Genre-Indicative Words:

In [74]:
texts = train_df['overview_clean']
labels = train_df['genre_list']

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_lr = TfidfVectorizer(max_features=15000, min_df=5)
X_lr = tfidf_lr.fit_transform(texts)

In [76]:
from sklearn.linear_model import LogisticRegression

indicative_words = {}

for i, genre in enumerate(mlb.classes_):

    y = Y_train[:, i]

    clf = LogisticRegression(max_iter=200)
    clf.fit(X_lr, y)

    coefs = clf.coef_[0]

    top_idx = np.argsort(coefs)[-10:]
    words = [tfidf_lr.get_feature_names_out()[j] for j in top_idx]

    indicative_words[genre] = words

In [77]:
for genre, words in indicative_words.items():
    print("\n", genre)
    print(words)


 Action
['cia', 'terrorist', 'target', 'fight', 'ruthless', 'hero', 'criminal', 'assassin', 'cop', 'agent']

 Adventure
['world', 'force', 'dinosaur', 'army', 'earth', 'power', 'jungle', 'bond', 'save', 'adventure']

 Animation
['penguin', 'dream', 'monster', 'city', 'animal', 'world', 'dragon', 'animated', 'human', 'adventure']

 Comedy
['decides', 'star', 'rock', 'big', 'girlfriend', 'friend', 'parent', 'show', 'guy', 'comedy']

 Crime
['money', 'killer', 'crime', 'criminal', 'detective', 'drug', 'fbi', 'cop', 'murder', 'police']

 Documentary
['moore', 'personal', 'tour', 'film', 'michael', 'interview', 'footage', 'filmmaker', 'look', 'documentary']

 Drama
['mother', 'yearold', 'war', 'relationship', 'drama', 'love', 'change', 'struggle', 'story', 'life']

 Family
['turtle', 'little', 'world', 'land', 'christmas', 'animal', 'save', 'boy', 'dog', 'adventure']

 Fantasy
['witch', 'century', 'king', 'world', 'magic', 'ancient', 'dragon', 'power', 'evil', 'vampire']

 Fiction
['power'